In [3]:
import findspark #Si estamos en linux esto no hace falta
findspark.init()#Si estamos en linux esto no hace falta
findspark.find()#Si estamos en linux esto no hace falta
import pyspark
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [4]:
#Antes de cargar fichero vamos a ver la configuración:
#si ejecutamoso sc, nos va a decir la version, 
sc

<SparkContext master=local[*] appName=pyspark-shell>

# Leer txt y crear RDD

In [5]:
#Creamos un fichero a mano y lo leemos
miprimer_rdd = sc.textFile('FicheroPrueba.txt')

In [6]:
miprimer_rdd.count()

4

In [5]:
miprimer_rdd.first()

'Linea 1'

In [6]:
miprimer_rdd.take(2)

['Linea 1', 'Linea 2']

# PARALELIZE
#Podemos crear RDD directamente desde aqui sin necesidad de leer de fichero, para ello vamos a usar parallelize, basicamente
lo que estamos haciendo es una colleccion de elemntos distribuida en nuestros clusters, entonces si por ejemplo tenemos una lista,normal, una vez le hagamos el parallelize, pasara a estar distribuida por nuestro sistema. mas adelante veremos que podemos elegir el numero de particiones que le queremos dar

In [8]:
miRDD = sc.parallelize([1,2,3,4,5,6,7,8,9])

In [9]:
miRDD #Vemos que ya tenemos nuestro RDD

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [15]:
lista  = ['Hola','Adios','Hello']
#Tenemos una lista normal y la queremos crear RDD:
lista_a_RDD = sc.parallelize(lista)
#Podemos elegir las particiones que queremos, aunq ya spark por defecto las asigna
lista_a_RDD = sc.parallelize(lista,4)


# TAKE

In [10]:
#Para coger X elementos del RDD, es similar  aun limit
miRDD.take(4)

[1, 2, 3, 4]

# COLLECT


In [12]:
#Si queremos que nos muestre todo podemos usar collect(), hay que llevar cuidado si es un RDD muy muy grande,
#Mejor no intentar mostrar todo, aun que los ejercicios que iremos haciendo no creo que haya problema, pero hay que
#tenerlo en cuenta.
miRDD.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [6]:
#Tambien podemos leer RDD de otras fuentes como por ejemplo de CSV

miprimer_rdd_CSV = sc.textFile('fichero_csv.csv')

In [7]:
#Del mismo modo podemos ver la informacion:
miprimer_rdd_CSV.collect()

['a,1,5,9', 'b,2,6,10', 'c,3,7,11', 'd,4,8,12']

# EVALUACION PEREZOSA

In [10]:
#EVALUACION PEREZOSA
#Si recordamos deciamos que Spark no va a ejecutar nada hasta
#que se aplique una accion sobre el RDD, una accion por ejemplo es take(2), queremos que nos muestre 2 elementos
#de nuestro RDD
#Si creamos un RDD a partir de un fichero que no existe, spark no me va a decir que no existe hasta que no intente
#hacer algo sobre ese RDD vamos a verlo:
miprimer_rdd = sc.textFile('UnFicheroQueNoExisteEnMiPc.txt')


In [11]:
miprimer_rdd

UnFicheroQueNoExisteEnMiPc.txt MapPartitionsRDD[6] at textFile at NativeMethodAccessorImpl.java:0

In [12]:
#Vemos que ha creado el RDD a partir de un fichero, pero este fichero realmente ni existe, pero como aun no ha intentado
#hacer nada sobre el fichero, spark aun no sabe que ese fichero no existe, cuando se va a dar cuenta? cuando le pida una 
#accion sobre el RDD
miprimer_rdd.take(2) #Vemos que ahora nos da error, cuando ha ido a procesar el RDD

Py4JJavaError: An error occurred while calling o71.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/C:/Users/fedev/Documents/Ejercicios Spark/Sesion 1/UnFicheroQueNoExisteEnMiPc.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:205)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.s

In [13]:
#Entocnes aqui cuando creabamos nuestro RDD a partir de fichero
miprimer_rdd = sc.textFile('FicheroPrueba.txt')

In [14]:
#Ene se momento aun no ha hecho nada
#Pero cuando le aplicamos un take(2), entonces ejecuta todo el DAG
miprimer_rdd.take(2)

['Linea 1', 'Linea 2']